In [1]:
from mycelia.shared.client import download_model, submit_model
from mycelia.shared.chain import *
from mycelia.shared.app_logging import configure_logging
from mycelia.shared.checkpoint import (
    get_resume_info,
)
from mycelia.shared.config import MinerConfig
import bittensor 


configure_logging()

config = MinerConfig()
config.model.device = "cuda:2"
config = MinerConfig.from_path("/home/isabella/crucible/subnet-MoE/checkpoints/miner/miner/hk1/foundation/config.yaml")
wallet, subtensor = setup_chain_worker(config)

2025-12-15 21:45:40 [warning  ] Cannot find the wallet key by name coldkey name: template_coldkey_name, hotkey name: template_hotkey_name, please make sure it has been set correctly if you are reloading from a  or use --hotkey_name & --coldkey_name when you are creating a config file from a template. [mycelia.shared.config] error=Generic error: Failed to get hotkey: FileNotFound("Keyfile at: /home/isabella/.bittensor/wallets/template_coldkey_name/hotkeys/template_hotkey_name does not exist.")


In [2]:
import requests

def ping_url(url, timeout=3):
    try:
        r = requests.get(url, timeout=timeout, allow_redirects=True)
        return r.text
    
    except requests.RequestException as e:
        return None

print(ping_url("http://localhost:8000/ping"))


{"status":"ok"}


In [12]:
url = "149.137.225.62:8001"
# url = "localhost:8000"
download_model(
    url=f"http://{url}/get-checkpoint",
    my_hotkey=wallet.hotkey,  # type: ignore
    target_hotkey_ss58="5DoHdXfDYraqPzkLjrXGMZxvGXYdDYhuC8tGbQdb4zvz2LbH",
    block=subtensor.block,
    expert_group_ids=[config.task.expert_group_id],
    token=getattr(config.cycle, "token", ""),
    out=config.ckpt.validator_checkpoint_path / f"uid{3}_{'hk3'}.pt",
)

Detected zip archive at /home/isabella/crucible/subnet-MoE/checkpoints/miner/validator_checkpoint/uid3_hk3.pt, extracting...
Extracted files to /home/isabella/crucible/subnet-MoE/checkpoints/miner/validator_checkpoint
Done:       22.00 B / 22.00 B (100.0%) in 0.0s @ 17.36 KiB/s


In [4]:
config.ckpt.validator_checkpoint_path

PosixPath('/home/isabella/crucible/subnet-MoE/checkpoints/miner/validator_checkpoint')

In [3]:
import time 

time.time()

1765834777.2508082

In [ ]:
from pathlib import Path

resume, start_step, latest_checkpoint_path = get_resume_info(rank=0, config=config)
submit_model(
    url="http://localhost:8000/submit-checkpoint",
    token="",
    step=0,
    uid=1,
    hotkey=config.chain.hotkey_ss58,
    model_path=Path(latest_checkpoint_path) / "model.pt",
)

2025-11-05 23:24:27 [info     ] rank 0: Latest checkpoint found in /home/isabella/crucible/subnet-MoE/checkpoints/miner/miner/hk1/foundation/inner_opt_step_7980 [mycelia.shared.checkpoint]


RuntimeError: Unexpected error during upload: Upload failed: HTTP 422: {'detail': [{'type': 'missing', 'loc': ['body', 'uid'], 'msg': 'Field required', 'input': None}]}